In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
path = "/Users/alexwang/Desktop/Datathon/data/demo_2011-2017.6.csv"
df_demo = pd.read_csv(path)


In [4]:
path_zip = '../data/gdp/ZIP-COUNTY-FIPS_2017-06.csv'
df_FZ = pd.read_csv(path_zip, sep=",", dtype=object)

In [6]:
df_FZ["ZIP_f3"] = np.nan

#convert zipcode to only first 3 zipcodes
def remove_last(ser):
    ser["ZIP_f3"] = ser["ZIP"][:3]+"**"
    return ser

df_FZ.apply(remove_last, axis=1)

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP,ZIP_f3
0,36003,Autauga County,AL,01001,H1,360**
1,36006,Autauga County,AL,01001,H1,360**
2,36067,Autauga County,AL,01001,H1,360**
3,36066,Autauga County,AL,01001,H1,360**
4,36703,Autauga County,AL,01001,H1,367**
...,...,...,...,...,...,...
52884,00850,St. Croix Island,VI,78010,H4,008**
52885,00840,St. Croix Island,VI,78010,H4,008**
52886,00820,St. Croix Island,VI,78010,H4,008**
52887,00830,St. John Island,VI,78020,H4,008**


In [7]:
# Add FIPS to our dataset
df_demo["FIPS"] = np.nan

def add_FIPS(ser):
    state = str(int(ser[1]))
    while len(state) < 2:
        state = "0"+state
    county = str(int(ser[2]))
    while len(county) < 3:
        county = "0"+county
    ser["FIPS"] = state+county
    return ser

df_demo = df_demo.apply(add_FIPS, axis=1)

In [8]:
# Drop unnecessary elements
df_demo = df_demo.drop(["SUMLEV", "STATE", "COUNTY"], axis=1)

In [9]:
# Drop YEAR 2 and 3
# Convert the rest to regular years
df_demo = df_demo.loc[(df_demo["YEAR"] != 2) & (df_demo["YEAR"] != 3)]

In [10]:
# Convert YEAR from 1, 2, 3, 4 to 2011...
def convert_year(ser):
    if ser["YEAR"] == 1:
        ser["YEAR"] = 2010
    elif ser["YEAR"] == 4:
        ser["YEAR"] = 2011
    elif ser["YEAR"] == 5:
        ser["YEAR"] = 2012
    elif ser["YEAR"] == 6:
        ser["YEAR"] = 2013
    elif ser["YEAR"] == 7:
        ser["YEAR"] = 2014
    elif ser["YEAR"] == 8:
        ser["YEAR"] = 2015
    elif ser["YEAR"] == 9:
        ser["YEAR"] = 2016
    elif ser["YEAR"] == 10:
        ser["YEAR"] = 2017
    return ser

df_demo = df_demo.apply(convert_year, axis=1)

In [18]:
# Combine different age groups
df_grouped = df_demo.groupby(["FIPS", "YEAR"]).sum()
df_grouped = df_grouped.drop(["AGEGRP"], axis=1)
df_grouped

TOT_POP  TOT_MALE  TOT_FEMALE  WA_MALE  WA_FEMALE  BA_MALE  \
FIPS  YEAR                                                               
01001 2010   109142     53138       56004    42590      44004     9118   
      2011   110398     53944       56454    42968      44378     9448   
      2012   109854     53636       56218    42642      43930     9464   
      2013   109390     53146       56244    42044      43792     9390   
      2014   109728     53570       56158    42152      43664     9626   
...             ...       ...         ...      ...        ...      ...   
56045 2013    14272      7526        6746     7148       6436       48   
      2014    14284      7546        6738     7144       6418       54   
      2015    14362      7588        6774     7186       6470       52   
      2016    14396      7562        6834     7130       6510       50   
      2017    13854      7268        6586     6808       6252       58   

            BA_FEMALE  IA_MALE  IA_FEMALE  AA_MALE  ...  HWAC_MALE  \
FIPS  YEAR                                          ...              
01001 2010      10260      238        278      400  ...       1214   
      2011      10308      236        284      416  ...       1250   
      2012      10430      244        308      418  ...       1248   
      2013      10564      260        300      452  ...       1222   
      2014      10590      252        306      482  ...       1264   
...               ...      ...        ...      ...  ...        ...   
56045 2013         30      134         96       36  ...        260   
      2014         36      134         98       52  ...        288   
      2015         28      126         86       66  ...        288   
      2016         28      130         82       84  ...        302   
      2017         30      138         88       98  ...        282   

            HWAC_FEMALE  HBAC_MALE  HBAC_FEMALE  HIAC_MALE  HIAC_FEMALE  \
FIPS  YEAR                                                                
01001 2010         1076        114           96         52           64   
      2011         1132        108           92         58           62   
      2012         1108        126           90         56           68   
      2013         1164        142          124         78           66   
      2014         1208        170          164         60           76   
...                 ...        ...          ...        ...          ...   
56045 2013          208          8           14         40           38   
      2014          216         14           14         40           34   
      2015          220          6           16         36           26   
      2016          236         12           18         48           38   
      2017          240         12           16         46           34   

            HAAC_MALE  HAAC_FEMALE  HNAC_MALE  HNAC_FEMALE  
FIPS  YEAR                                                  
01001 2010         18           22         38           20  
      2011         18           30         34           18  
      2012         26           18         42           16  
      2013         38           26         32           16  
      2014         30           28         34           24  
...               ...          ...        ...          ...  
56045 2013          6            4          2            2  
      2014          4            2          2            0  
      2015          6            6          8            6  
      2016          4            0          2            2  
      2017          4           12          4            2  

[25136 rows x 73 columns]

In [ ]:
# Change population to percentage
def conv_pop_to_perc(ser):
    ser[1:] = pd.Series((ser[1:].astype('float64')).to_numpy()/float(ser[0]))
    return ser

df_grouped = df_grouped.apply(conv_pop_to_perc, axis=1)


In [13]:
df_grouped.reset_index()

,FIPS,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,01001,2010,109142.0,0.486870,0.513130,0.390226,0.403181,0.083543,0.094006,0.002181,...,0.011123,0.009859,0.001045,0.000880,0.000476,0.000586,0.000165,0.000202,0.000348,0.000183
1,01001,2011,110398.0,0.488632,0.511368,0.389210,0.401982,0.085581,0.093371,0.002138,...,0.011323,0.010254,0.000978,0.000833,0.000525,0.000562,0.000163,0.000272,0.000308,0.000163
2,01001,2012,109854.0,0.488248,0.511752,0.388170,0.399894,0.086151,0.094944,0.002221,...,0.011361,0.010086,0.001147,0.000819,0.000510,0.000619,0.000237,0.000164,0.000382,0.000146
3,01001,2013,109390.0,0.485840,0.514160,0.384350,0.400329,0.085840,0.096572,0.002377,...,0.011171,0.010641,0.001298,0.001134,0.000713,0.000603,0.000347,0.000238,0.000293,0.000146
4,01001,2014,109728.0,0.488207,0.511793,0.384150,0.397929,0.087726,0.096511,0.002297,...,0.011519,0.011009,0.001549,0.001495,0.000547,0.000693,0.000273,0.000255,0.000310,0.000219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25131,56045,2013,14272.0,0.527326,0.472674,0.500841,0.450953,0.003363,0.002102,0.009389,...,0.018217,0.014574,0.000561,0.000981,0.002803,0.002663,0.000420,0.000280,0.000140,0.000140
25132,56045,2014,14284.0,0.528283,0.471717,0.500140,0.449314,0.003780,0.002520,0.009381,...,0.020162,0.015122,0.000980,0.000980,0.002800,0.002380,0.000280,0.000140,0.000140,0.000000
25133,56045,2015,14362.0,0.528339,0.471661,0.500348,0.450494,0.003621,0.001950,0.008773,...,0.020053,0.015318,0.000418,0.001114,0.002507,0.001810,0.000418,0.000418,0.000557,0.000418
25134,56045,2016,14396.0,0.525285,0.474715,0.495276,0.452209,0.003473,0.001945,0.009030,...,0.020978,0.016393,0.000834,0.001250,0.003334,0.002640,0.000278,0.000000,0.000139,0.000139


In [14]:
# df_grouped.to_csv('demographics.csv')